In [6]:

#if possible every update function need a way to skip the files that have already been integrated
# for the export and the read in, although this might be a bit of an optimization

import os

In [10]:
exportDataPath = "/home/chowder/Documents/dataExports/fitbit/25-1-7/takeout-20250106T144047Z-001/Takeout/Fitbit/Global Export Data/"
dir_list = os.listdir(exportDataPath)
hrFilenames = sorted([x for x in dir_list if x.split("-")[0] == "heart_rate"])
hrFilenames[-1]
# no reading yet we can test that later

'heart_rate-2025-01-05.json'

In [11]:
#takes like 50 seconds to make 17 million rows
import json
import pytz
from datetime import datetime


def to_iso(s):
    return '20' + s[6:8] + '-' + s[0:2] + '-' + s[3:5] + s[8:]

samplesCount = 0
samplesList = []
for fn in hrFilenames:
    data = json.load(open(exportDataPath + fn))
    for row in data:
        sampleDT = datetime.fromisoformat(to_iso(row["dateTime"]))
        samplesList.append([sampleDT, 
                            row["value"]["confidence"], 
                            row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 1_000_000 == 0:
            print(f"added {samplesCount} samples so far")
print(f"added {samplesCount} samples")

added 1000000 samples so far
added 2000000 samples so far
added 3000000 samples so far
added 4000000 samples so far
added 5000000 samples so far
added 6000000 samples so far
added 7000000 samples so far
added 8000000 samples so far
added 9000000 samples so far
added 10000000 samples so far
added 11000000 samples so far
added 12000000 samples so far
added 13000000 samples so far
added 14000000 samples so far
added 15000000 samples so far
added 16000000 samples so far
added 17000000 samples so far
added 17335490 samples


In [12]:
import pandas as pd

columnNames = ["sampleDT", "confidence", "value"]
samplesList = sorted(samplesList, key=lambda x: x[0]) #sort by timestamp

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)
fitbitHRdf = fitbitHRdf.set_index("sampleDT")
fitbitHRdf.index = fitbitHRdf.index.tz_localize('UTC')
fitbitHRdf.index = pd.to_datetime(fitbitHRdf.index)

fitbitHRdf["confidence"] = fitbitHRdf["confidence"].astype('uint8')
fitbitHRdf["value"] = fitbitHRdf["value"].astype('uint8')
fitbitHRdf.dtypes

/home/chowder/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


confidence    uint8
value         uint8
dtype: object

In [13]:
#now let's redo the writeWorkingTSDf
#ngl I like the short hashes


In [14]:
import hashlib
import pickle

# Function to compute a short hash of a Python object
def short_hash(obj, length=8):
    # Serialize the object using pickle
    obj_bytes = pickle.dumps(obj)
    
    # Compute MD5 hash of the serialized object
    hash_obj = hashlib.md5(obj_bytes)
    
    # Return the hash truncated to the specified length
    return hash_obj.hexdigest()[:length]

In [15]:
# this writes a file for a timeSeries of a DF
def writeTimeSeriesDf(TSDf, targetPath):
    sh = short_hash(TSDf)
    parquetName = TSDf.iloc[0].name.strftime('%Y-%m-%dT%H%M%S%z') +\
                "_" +\
                TSDf.iloc[-1].name.strftime('%Y-%m-%dT%H%M%S%z') +\
                "_" + sh + "_" + ".parquet.gzip"
    print(f"saved to a file named {parquetName}")

    TSDf.to_parquet(targetPath + parquetName,
            compression='gzip') 

In [16]:
#takes in a dataframe you want to save and does it in multiple files
#with the number of rows between rows_per_file and 2 * rows_per_file 
def saveRows(TSDf, targetPath, rows_per_file):
    if len(TSDf) == 0: return
    startRow = 0
    endRow = len(TSDf)
    rows_remaining = endRow - startRow
    while rows_remaining > 2 * rows_per_file:
        print(f'{rows_remaining} is too many rows writing {startRow} to {(endRow - rows_remaining) + rows_per_file}')
        writeTimeSeriesDf(TSDf.iloc[startRow: (endRow - rows_remaining) + rows_per_file + 1], targetPath)
        rows_remaining -= rows_per_file
        startRow += rows_per_file
    writeTimeSeriesDf(TSDf.iloc[startRow:endRow+1], targetPath)

In [17]:
#given a df returns the approximate number of rows to get a target file size 
def calcRowsPerFile(Df, targetFileSize, targetPath, fileName = 'test.parquet.gzip'):
    if fileName == 'test.parquet.gzip':
        fileRows = 1_000_000
        if len(Df) < fileRows: fileRows = len(Df)-1
        Df.iloc[:fileRows].to_parquet(targetPath + fileName,
                        compression='gzip')
        file_size = os.path.getsize(targetPath + fileName)
        os.remove(targetPath + fileName)
    else:
        fileRows = len(pd.read_parquet(targetPath + fileName))
        file_size = os.path.getsize(targetPath + fileName)
    
    rows_per_file = int(fileRows//(file_size/targetFileSize))
    return rows_per_file

In [18]:
#iterates over exiting files and adds rows in the relevant scope
def writeToExistingTSFiles(TSDf, fileNames, targetPath, rows_per_file):
    tzi = TSDf.index[0].tzinfo
    for fileNum, fileName in enumerate(fileNames):
        if fileNum == 0:
            startTime = TSDf.index[0]
        else:
            startTime = pd.to_datetime(fileName.split('_')[0]).tz_convert(tzi)
        
        if len(fileNames) == 1 or fileNum == len(fileNames) - 1:
            endTime = TSDf.index[-1]
        else:
            endTime = pd.to_datetime(fileNames[fileNum + 1].split('_')[0]).tz_convert(tzi)
        
        # if the hash doesn't match write a new file
        if short_hash(TSDf.loc[startTime:endTime]) != fileName.split('_')[2]:
            print("the hashes don't match")
            os.remove(targetPath + fileName)
            saveRows(TSDf.loc[startTime:endTime], targetPath, rows_per_file)
        else:
            print(f'hashes match for {fileName}')

In [19]:
#this may be the only function I need to change
# the features I want to add is to 
# take in partyName, deviceName, dataType, dataSource
# convert the DF to UTC

#TODO: rename the wordkingData file OldWorkingData
#I should make the update/impot function be the one that handles diffrent sensors

In [26]:

def writeWorkingTSDf(partyName, deviceName, dataType, dataSource, TSDf, targetFileSize = 2 * 1024 * 1024):
    dataFolderName = "_".join([partyName, deviceName, dataType, dataSource]) + "/"
    fullPath = workingDataPath + dataFolderName
    if not os.path.exists(fullPath):
        os.makedirs(fullPath)
    currentFileNames = sorted(os.listdir(fullPath))

    TSDf.index = TSDf.index.tz_convert('UTC')


    if len(currentFileNames) == 0:
        rows_per_file = calcRowsPerFile(TSDf, targetFileSize, fullPath)
        saveRows(TSDf, fullPath, rows_per_file)

    else:
        rows_per_file = calcRowsPerFile(TSDf, targetFileSize, fullPath, currentFileNames[0])
        writeToExistingTSFiles(TSDf, currentFileNames, fullPath, rows_per_file)


In [27]:
workingDataPath = "/home/chowder/Documents/workingData/"
writeWorkingTSDf("fitbit", "charge4or5", "hr", "builtin", fitbitHRdf)


17335490 is too many rows writing 0 to 365451
saved to a file named 2020-05-13T170219+0000_2020-06-16T005314+0000_100a0466_.parquet.gzip
16970039 is too many rows writing 365451 to 730902
saved to a file named 2020-06-16T005314+0000_2020-07-19T174401+0000_4818a063_.parquet.gzip
16604588 is too many rows writing 730902 to 1096353
saved to a file named 2020-07-19T174401+0000_2020-08-22T082256+0000_4464f29d_.parquet.gzip
16239137 is too many rows writing 1096353 to 1461804
saved to a file named 2020-08-22T082256+0000_2020-09-25T171817+0000_573f9dcc_.parquet.gzip
15873686 is too many rows writing 1461804 to 1827255
saved to a file named 2020-09-25T171817+0000_2020-10-29T013912+0000_bd11fc15_.parquet.gzip
15508235 is too many rows writing 1827255 to 2192706
saved to a file named 2020-10-29T013912+0000_2020-12-01T093854+0000_91dcda5b_.parquet.gzip
15142784 is too many rows writing 2192706 to 2558157
saved to a file named 2020-12-01T093854+0000_2021-01-04T133306+0000_992b3849_.parquet.gzip
14

In [ ]:
# wooo the write  looks like it's working!
# now I'll do the writes for apple and polar
# none of these dataframes are so big that we can't just
# read it in, add to it, sort it, deduplicate it, write it back for updates

In [ ]:

import hashlib
import pickle

# Function to compute a short hash of a Python object
def short_hash(obj, length=8):
    # Serialize the object using pickle
    obj_bytes = pickle.dumps(obj)
    
    # Compute MD5 hash of the serialized object
    hash_obj = hashlib.md5(obj_bytes)
    
    # Return the hash truncated to the specified length
    return hash_obj.hexdigest()[:length]

# this writes a file for a subset of a DF
def writeHRDfFile(HRDf, workingDataHRPath):
    sh = short_hash(HRDf)
    parquetName = HRDf.iloc[0].name.strftime('%Y-%m-%dT%H%M%S%z') +\
                "_" +\
                HRDf.iloc[-1].name.strftime('%Y-%m-%dT%H%M%S%z') +\
                "_" + sh + "_" + ".parquet.gzip"
    print(f"saved to a file named {parquetName}")

    HRDf.to_parquet(workingDataHRPath + parquetName,
            compression='gzip') 

#takes in a dataframe you want to save and does it in multiple files
def saveRows(df, workingDataHRPath, rows_per_file):
    if len(df) == 0: return
    startRow = 0
    endRow = len(df)
    rows_remaining = endRow - startRow
    while rows_remaining > 2 * rows_per_file:
        print(f'{rows_remaining} is too many rows writing {startRow} to {(endRow - rows_remaining) + rows_per_file}')
        writeHRDfFile(df.iloc[startRow: (endRow - rows_remaining) + rows_per_file + 1], workingDataHRPath)
        rows_remaining -= rows_per_file
        startRow += rows_per_file
    writeHRDfFile(df.iloc[startRow:endRow+1], workingDataHRPath)

def rowsPerFile(Df, targetFileSize, workingDataHRPath, fileName = 'test.parquet.gzip'):
    if fileName == 'test.parquet.gzip':
        fileRows = 1_000_000
        if len(Df) < fileRows: fileRows = len(Df)-1
        Df.iloc[:fileRows].to_parquet(workingDataHRPath + fileName,
                        compression='gzip')
        file_size = os.path.getsize(workingDataHRPath + fileName)
        os.remove(workingDataHRPath + fileName)
    else:
        fileRows = len(pd.read_parquet(workingDataHRPath + fileName))
        file_size = os.path.getsize(workingDataHRPath + fileName)
    
    rows_per_file = int(fileRows//(file_size/targetFileSize))
    return rows_per_file

def writeToExistingHRFiles(HRDf, fileNames, workingDataHRPath, rows_per_file):
    tzi = HRDf.index[0].tzinfo
    for fileNum, fileName in enumerate(fileNames):
        if fileNum == 0:
            startTime = HRDf.index[0]
        else:
            startTime = pd.to_datetime(fileName.split('_')[0]).tz_convert(tzi)
        
        if len(fileNames) == 1 or fileNum == len(fileNames) - 1:
            endTime = HRDf.index[-1]
        else:
            endTime = pd.to_datetime(fileNames[fileNum + 1].split('_')[0]).tz_convert(tzi)
        
        # if the hash doesn't match write a new file
        if short_hash(HRDf.loc[startTime:endTime]) != fileName.split('_')[2]:
            print("the hashes don't match")
            os.remove(workingDataHRPath + fileName)
            saveRows(HRDf.loc[startTime:endTime], workingDataHRPath, rows_per_file)
        else:
            print(f'hashes match for {fileName}')

def writeWorkingHRDfParquet(deviceName, HRDf, clearFiles = True, targetFileSize = 2 * 1024 * 1024):
    workingDataHRPath = workingDataPath + deviceName + "/hr/"
    fileNames = sorted(os.listdir(workingDataHRPath))

    if len(fileNames) == 0:
        rows_per_file = calcRowsPerFile(HRDf, targetFileSize, workingDataHRPath)
        saveRows(HRDf, workingDataHRPath, rows_per_file)

    else:
        rows_per_file = calcRowsPerFile(HRDf, targetFileSize, workingDataHRPath, fileNames[0])
        writeToExistingHRFiles(HRDf, fileNames, workingDataHRPath, rows_per_file)

